In [16]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble  import RandomForestClassifier
from hyperopt import fmin, tpe, hp, rand, STATUS_OK, Trials
import warnings
warnings.filterwarnings("ignore")

# Define Configuration Space

In [10]:
#Define Search Space
HPOspace = hp.choice('classifier_type', [
    {
        #Define Search Space for Support Vector Machine
        'name': 'SVM',
        'C': hp.uniform('C', 1e-2, 100),
        'kernel': hp.choice('kernel', ["linear","rbf", "poly", "sigmoid"]),
        "coef0": hp.uniform('coef0', 0.0, 10.0),
        "degree": hp.choice("degree", range(2, 50)),
        "gamma": hp.uniform('gamma',0, 20),
    },
    {
        # Define Search Space for Random Forest
        'name': 'RF',
        'n_estimators': hp.choice("n_estimators", range(5, 100)),
        'criterion': hp.choice('criterion', ["gini", "entropy"]),
        'max_depth': hp.choice("max_depth", range(10, 200)),
        'max_features': hp.choice('max_features',['auto', 'sqrt', 'log2'])
        
    },
    ])

# Objective Function

In [14]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

def obj_func(params):
    params = {k : params[k] for k in params if params[k]}
    #print(params)
    classifier = params['name']
    params.pop("name", None)    
    #print(params)
    clf = SVC()
    if(classifier == 'SVM'):
        clf = SVC(**params)
    elif(classifier == 'RF'):
        clf = RandomForestClassifier(**params)
    mean= cross_val_score(clf, X, y).mean()
    loss = 1 - mean
    #print (mean)
    return loss

# Run Optimise

In [17]:
#Capturing Info with Trials
trials = Trials()
best_candidate = fmin(obj_func,HPOspace,algo=tpe.suggest, max_evals=50, trials=trials)

100%|██████████| 50/50 [00:13<00:00,  3.67trial/s, best loss: 0.019607843137254832]
